In [1]:
!curl http://localhost:9200/

{
  "name" : "node-1",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "Hd5owN6ERfSiT3DlTZTENg",
  "version" : {
    "number" : "8.2.2",
    "build_flavor" : "default",
    "build_type" : "deb",
    "build_hash" : "9876968ef3c745186b94fdabd4483e01499224ef",
    "build_date" : "2022-05-25T15:47:06.259735307Z",
    "build_snapshot" : false,
    "lucene_version" : "9.1.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [2]:
import elasticsearch 

In [3]:
es = elasticsearch.Elasticsearch('http://127.0.0.1:9200/')

### 1. Match query 

In [4]:
# Search documents in Elasticsearch
query={'match':{
          'text':'ebola'}
        }

hits = es.search(index='pandemic_docs', query=query, size=5) 
hits['hits']

{'total': {'value': 11, 'relation': 'eq'},
 'max_score': 6.931862,
 'hits': [{'_index': 'pandemic_docs',
   '_id': '5okCVIEB5DPWiFiNugmg',
   '_score': 6.931862,
   '_ignored': ['text.keyword'],
   '_source': {'article_title': 'Science diplomacy and pandemics',
    'section_title': 'Ebola',
    'main_section': 'Ebola',
    'section_number': 2,
    'page_id': 63936969,
    'text': "Science diplomacy and pandemics,Ebola\nThe Western African Ebola virus epidemic took more than 11,000 lives. Science diplomats traveled the world and worked with scientists from other countries in the hope of finding a vaccine to conquer the Ebola outbreak. The Ebola virus causes severe bleeding causing organ failure, lack of oxygen to the body, and it can lead to death. Scientists first discovered the Ebola virus in 1976 in Democratic Republic of Congo. Since then it has spread to other countries. It affected mostly West African countries. The largest outbreak of the Ebola virus from 2014 to 2016. There were

## 2. Term Query

In [5]:
query_term={'term':{
              'text':{
                  'value':'assessment', 
                   'boost':1.0}
                }
            }

hits = es.search(index='pandemic_docs', query=query_term, size=5) 
hits['hits']

{'total': {'value': 10, 'relation': 'eq'},
 'max_score': 5.759901,
 'hits': [{'_index': 'pandemic_docs',
   '_id': 'yIkCVIEB5DPWiFiNugmg',
   '_score': 5.759901,
   '_source': {'article_title': 'Pandemic severity index',
    'section_title': 'See also',
    'main_section': 'See also',
    'section_number': 5,
    'page_id': 9291245,
    'text': 'Pandemic severity index,See also\nPandemic Severity Assessment Framework\n2009 H1N1 influenza pandemic\nEarly Warning and Response System\nWHO pandemic phases\n\n\n== References ==',
    'source_url': 'https://en.wikipedia.org/wiki/Pandemic_severity_index',
    'tags': 'Pandemic severity index,See also'}},
  {'_index': 'pandemic_docs',
   '_id': 'GokCVIEB5DPWiFiNugmd',
   '_score': 4.7102184,
   '_ignored': ['text.keyword'],
   '_source': {'article_title': 'Cholera',
    'section_title': 'Surveillance',
    'main_section': 'Prevention',
    'section_number': 10,
    'page_id': 7591,
    'text': 'Cholera,Prevention,Cause,Surveillance\nSurveillan

### 3. Boolean Query

In [6]:
# Boolean search - `World Health Organziation` with match; filter out `External links`
query={'bool':{
          'must': {'match': {
                      'text':'World Health Organization'}
                      },
          'must_not': {'match': {
                        'section_title.keyword': 'External links'}
                    }
          }
}  
hits = es.search(index='pandemic_docs', query=query, size=5) 
hits['hits']

{'total': {'value': 158, 'relation': 'eq'},
 'max_score': 7.7753215,
 'hits': [{'_index': 'pandemic_docs',
   '_id': 'eokCVIEB5DPWiFiNugme',
   '_score': 7.7753215,
   '_ignored': ['text.keyword'],
   '_source': {'article_title': 'Johns Hopkins Center for Health Security',
    'section_title': 'Summary',
    'main_section': 'Summary',
    'section_number': 0,
    'page_id': 11008433,
    'text': 'Johns Hopkins Center for Health Security,Summary\nThe Johns Hopkins Center for Health Security (abbreviated CHS; previously the UPMC Center for Health Security, the Center for Biosecurity of UPMC, and the Johns Hopkins Center for Civilian Biodefense Strategies) is an independent, nonprofit organization of the Johns Hopkins Bloomberg School of Public Health, and part of the Environmental Health and Engineering department. It is concerned with the areas of health consequences from epidemics and disasters as well as averting biological weapons development, and implications of biosecurity for the 

In [7]:
# Boolean Search - match `Spanish Flu` and boost `Summary` section
query={'bool':{
          'must': [
              {'match': {
                   'text':'Spanish flu'}
                   },
              {'match':{
                    'main_section': {
                            'query':'Summary','boost': '5'}
                            }
                     }
              ]
       }
}
    
hits = es.search(index='pandemic_docs', query=query, size=5) 
hits['hits']

{'total': {'value': 3, 'relation': 'eq'},
 'max_score': 24.675499,
 'hits': [{'_index': 'pandemic_docs',
   '_id': '94kCVIEB5DPWiFiNugmg',
   '_score': 24.675499,
   '_ignored': ['text.keyword'],
   '_source': {'article_title': 'Swine influenza',
    'section_title': 'Summary',
    'main_section': 'Summary',
    'section_number': 0,
    'page_id': 1516915,
    'text': 'Swine influenza,Summary\nSwine influenza is an infection caused by any one of several types of swine influenza viruses. Swine influenza virus (SIV) or swine-origin influenza virus (S-OIV) is any strain of the influenza family of viruses that is endemic in pigs. As of 2009, the known SIV strains include influenza C and the subtypes of  influenza A known as H1N1, H1N2, H2N1, H3N1, H3N2, and H2N3.\nSwine influenza virus is common throughout pig populations worldwide. Transmission of the virus from pigs to humans is not common and does not always lead to human flu, often resulting only in the production of antibodies in the 

### 4. Highlight

In [7]:
body={'query':{
          'bool':{
              'must': {'match': {
                            'text':'ebola'}
                      },
              'must_not': {'match': {
                            'section_title.keyword': 'External links'}
                        },
              }
        },
            'highlight': {
                 'fields': {
                     'text': {},
                     }
            }
    }

hits = es.search(index='pandemic_docs', body=body, size=5) 
hits['hits']

/tmp/ipykernel_133358/2214272207.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  hits = es.search(index='pandemic_docs', body=body, size=5)


{'total': {'value': 11, 'relation': 'eq'},
 'max_score': 6.931862,
 'hits': [{'_index': 'pandemic_docs',
   '_id': '5okCVIEB5DPWiFiNugmg',
   '_score': 6.931862,
   '_ignored': ['text.keyword'],
   '_source': {'article_title': 'Science diplomacy and pandemics',
    'section_title': 'Ebola',
    'main_section': 'Ebola',
    'section_number': 2,
    'page_id': 63936969,
    'text': "Science diplomacy and pandemics,Ebola\nThe Western African Ebola virus epidemic took more than 11,000 lives. Science diplomats traveled the world and worked with scientists from other countries in the hope of finding a vaccine to conquer the Ebola outbreak. The Ebola virus causes severe bleeding causing organ failure, lack of oxygen to the body, and it can lead to death. Scientists first discovered the Ebola virus in 1976 in Democratic Republic of Congo. Since then it has spread to other countries. It affected mostly West African countries. The largest outbreak of the Ebola virus from 2014 to 2016. There were